In [1]:
import pandas as pd
from sklearn import preprocessing
import numpy as np

In [ ]:
postulaciones = pd.read_csv('Sources/postulaciones_for_training.csv').drop('Unnamed: 0', axis=1)
postulantes = pd.read_csv('Sources/postulantes_proc.csv').drop('Unnamed: 0', axis=1)
avisos = pd.read_csv('Sources/fiuba_6_avisos_detalle_limpio.csv').drop('Unnamed: 0', axis=1)
test = pd.read_csv('Sources/test_final_100k.csv')
visitas = pd.read_csv('Sources/visitas_proc.csv').drop('Unnamed: 0', axis=1)
preds = pd.DataFrame()

In [ ]:
postulaciones = postulaciones.merge(postulantes, on='idpostulante')
postulaciones = postulaciones.merge(avisos, on='idaviso')
postulaciones = postulaciones.merge(visitas, on=['idaviso','idpostulante'], how='left')
media_visitas = np.nanmean(postulaciones.loc[postulaciones['sepostulo']==1,'visito'])
postulaciones.loc[postulaciones['sepostulo']==1,'visito'].fillna(media_visitas,inplace=True)
resultados = postulaciones['sepostulo']
postulaciones.drop(['titulo','descripcion','idaviso','idpostulante','sepostulo'],axis=1, inplace=True)
postulaciones.head()

In [ ]:
test = test.merge(postulantes, on='idpostulante')
test = test.merge(avisos, on='idaviso')
test = test.merge(visitas, on=['idaviso','idpostulante'], how='left')
test.drop(['titulo','descripcion','idaviso','idpostulante','id'],axis=1, inplace=True)
test.head()

In [ ]:
test = pd.concat([test,postulaciones])

In [ ]:
test['fechanacimiento'].fillna(np.nanmean(test['fechanacimiento']),inplace=True)
test['FEM'].fillna(np.nanmean(test['FEM']),inplace=True)
test['MASC'].fillna(np.nanmean(test['MASC']),inplace=True)
test['NO_DECLARA'].fillna(np.nanmean(test['NO_DECLARA']),inplace=True)
test['prom_edad'].fillna(np.nanmean(test['prom_edad']),inplace=True)
test['FEM'].fillna(np.nanmean(test['FEM']),inplace=True)
lista = ['sabados', 'domingos','ingles', '3 anos', '2 anos', '4 anos', '5 anos', 'visito']
for i in lista:
    test[i].fillna(0,inplace=True)

In [ ]:
test.head()

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for col in test.columns:
    if not test[col].dtype == 'O': continue
    test[col] = test[col].fillna('na')
    le.fit(test[col])
    test[col] = le.transform(test[col])

In [ ]:
postulaciones = test[100000:].fillna(-1)
test = test[:100000].fillna(-1)

In [ ]:
partitions = []
partitions_res = []
n = 5
index = int(postulaciones.shape[0]/n)
for i in range(n):
    partitions.append(postulaciones[index*i:index*(i+1)])
    partitions_res.append(resultados[index*i:index*(i+1)])


In [ ]:
aviso = None
postulantes = None
visitas = None

In [ ]:
def proba(result):
    prob_1 = []
    for x in result:
        prob_1.append(x[1])
    return prob_1

In [ ]:
print('Arranca XG')

import xgboost as xgb
dtrain = xgb.DMatrix(data=partitions[0],label=partitions_res[0])
dtest = xgb.DMatrix(data=partitions[4])
param = {'max_depth':20, 'eta':1, 'silent':1, 'objective':'binary:logistic', 'subsample':0.8}
num_round = 50
bst = xgb.train(param, dtrain, num_round)
preds['XGB'] = bst.predict(dtest)

print('Termino XG')

In [ ]:
print('Arranca RF')

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
clf=RandomForestClassifier(n_estimators=50, warm_start=True, criterion='entropy', n_jobs=-1)
clf.fit(partitions[1],partitions_res[1])
preds['RF'] = proba(clf.predict_proba(partitions[4]))

print('Termino RF')

In [ ]:
print('Arranca NB')

from sklearn.naive_bayes import BernoulliNB
nb = BernoulliNB()
nb.fit(partitions[2] ,partitions_res[2])
BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)
preds['NB'] = proba(nb.predict_proba(partitions[4]))

print('Termino NB')

In [ ]:
print('Arranca PERCEPTRON')

from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(partitions[3], partitions_res[3])
preds['PERCEPTRON'] = proba(clf.predict_proba(partitions[4]))

print('Termino PERCEPTRON')

In [ ]:
print('Entreno KNN')

from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=25 ,n_jobs=-1, algorithm='auto')
neigh.fit(preds, partitions_res[4])

print('KNN ya esta entrenado')



In [ ]:
preds.head()

In [ ]:
final = pd.DataFrame()

In [ ]:
print('Arranca XGB final')

aviso = None
postulantes = None
import xgboost as xgb
dtrain = xgb.DMatrix(data=postulaciones, label=resultados)
dtest = xgb.DMatrix(data=test)
param = {'max_depth':20, 'eta':1, 'silent':1, 'objective':'binary:logistic', 'subsample':0.8}
num_round = 50
final['XGB'] = xgb.train(param, dtrain, num_round)

print('Termino XGB final')

In [ ]:
print('Arranca RF final')

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
clfr=RandomForestClassifier(n_estimators=50, warm_start=True, criterion='entropy', n_jobs=-1)
final['RF'] = clfr.fit(postulaciones,resultados)

print('Termino RF final')

In [ ]:
print('Arranca NB final')

from sklearn.naive_bayes import BernoulliNB
nb = BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)
nb.fit(postulaciones, resultados)
final['NB'] = proba(nb.predict_proba(test))

print('Termino NB final')

In [ ]:
print('Arranca PPERCEPTRON final')
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(postulaciones, resultados)
final['PERCEPTRON'] = proba(clf.predict_proba(test))

print('Termino PERCEPTRON final')

In [ ]:
final.head()

In [ ]:
print('Predigo con KNN')

resultado = proba(neigh.predict(final))

print('Termino KNN')

In [ ]:
test = pd.read_csv('Sources/test_final_100k.csv')
test['sepostulo'] = resultado
test.drop(['idaviso','idpostulante'],axis=1,inplace=True)
test.to_csv('Sources/resultado_final_pipe.csv', index=False)